In [251]:
import pandas as pd
import numpy as np


In [16]:
with open('transactions_small.csv') as f:
    print(f)

<_io.TextIOWrapper name='transactions_small.csv' mode='r' encoding='cp1251'>


Загружаем файлы с транзакциями и клиентами

In [249]:
df_clients = pd.read_csv("./clients_small.csv", sep=';')
df_transactions = pd.read_csv("./transactions_small.csv", sep=';')


In [8]:
df_clients.head(3)

,customer_id,card_id,region_name,sex,birth_date
0,02F4DB34C5696A668068CA3C1DE7C2D5,6600B4FDA036A6F00D464541EAD3B21F,Москва,F,1938-08-18
1,0308713D02D676920F2961E2A84B4516,1FB9344B8661069910D184EF19C448C7,Сахалинская область,F,1990-03-26
2,032672FDC2CF85213079F83E447AF75B,4C8AD60064AF412753EB54EAFCD8C59D,Самарская область,F,1989-01-13


In [17]:
df_transactions.head(3)

,customer_id,card_id,tr_datetime,tr_normalized_amount,currency,mcc,merchant_name,merchant_city
0,634E7564849D27B3C452229E405629A6,52CD1B87531555F5611C663562FC686D,2019-09-07,10.00,810,5999,i-bank ┴шыaщэ >moscow ru,moscow
1,B20A638E7EB926C63102144596F38C71,9D4F087597B65028953FBE8A8FBF48DE,2019-09-20,313.04,810,5411,dixy moskva ru,moscow
2,125EBE0A38873D4888408F85877282EF,B8859E8DB421C73B785521A6252A36CB,2019-09-23,250.00,810,5999,i-bank teыe 2 >moscow ru,moscow


Пробуем найти косячные данные типа "i-bank ┴шыaщэ" в столбце merchant_name \
Для этого создаем функцию, которая находит не Unicode символы 

In [24]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [29]:
df_transactions['merchant_name_eng_name'] = df_transactions.apply(lambda x: isEnglish(x['merchant_name']), axis=1)

Исследуем merchant_name которые на распознались и оцениваем их количество в общей выборке

In [38]:
df_transactions[df_transactions['merchant_name_eng_name'] == False ]['merchant_name'].unique()

array(['i-bank ┴шыaщэ         >moscow         ru',
       'i-bank teыe 2         >moscow         ru',
       'i-bank meуa╘oэ        >moscow         ru',
       'i-bank meгaфoн        >moscow         ru',
       'i-bank билaйн         >moscow         ru',
       'i-bank ╪ъoы№эoe яшєaэш>moscow         ru',
       'i-bank teлe 2         >moscow         ru',
       'i-bank m├tc           >moscow         ru',
       'i-bank ─om.ru         >moscow         ru',
       'i-bank teыe 2 (aтєoяыa>moscow         ru',
       'i-bank yota. coєoтa\xa0 c>moscow         ru',
       'i-bank mгtc           >moscow         ru',
       'i-bank mtc (aтєoяыaєeц>moscow         ru',
       'i-bank moєшт          >moscow         ru',
       'i-bank pocєeыeъoь ╧ыaє>moscow         ru',
       'i-bank tpшъoыop tb    >moscow         ru',
       'i-bank ┴шыaщэ (aтєoяыa>moscow         ru',
       'i-bank aшcє           >moscow         ru',
       'i-bank дom.ru         >moscow         ru',
       'i-bank mtc фикcиpoвa

In [48]:
bad_merchants =  df_transactions[df_transactions['merchant_name_eng_name'] == False ]['merchant_name'].shape[0]
good_merchants =  df_transactions['merchant_name'].shape[0]
print('good_merchants : ',good_merchants,'| bad_merchants : ',bad_merchants,'| bad_merchants_percent : ', 100*bad_merchants/good_merchants)

good_merchants :  623956 | bad_merchants :  15670 | bad_merchants_percent :  2.5113950342652367


Процент не такой большой, и количество таких непонятных мерчантов не такой большое. \
Исследуем их глазами на причастность к кофе по названию компании \
Ничего похожего на кофе не находим, поэтому просто выкидыаем их из аналитики 

In [155]:
df_transactions[df_transactions['merchant_name_eng_name'] == False ]['merchant_name'].unique()

array(['i-bank ┴шыaщэ         >moscow         ru',
       'i-bank teыe 2         >moscow         ru',
       'i-bank meуa╘oэ        >moscow         ru',
       'i-bank meгaфoн        >moscow         ru',
       'i-bank билaйн         >moscow         ru',
       'i-bank ╪ъoы№эoe яшєaэш>moscow         ru',
       'i-bank teлe 2         >moscow         ru',
       'i-bank m├tc           >moscow         ru',
       'i-bank ─om.ru         >moscow         ru',
       'i-bank teыe 2 (aтєoяыa>moscow         ru',
       'i-bank yota. coєoтa\xa0 c>moscow         ru',
       'i-bank mгtc           >moscow         ru',
       'i-bank mtc (aтєoяыaєeц>moscow         ru',
       'i-bank moєшт          >moscow         ru',
       'i-bank pocєeыeъoь ╧ыaє>moscow         ru',
       'i-bank tpшъoыop tb    >moscow         ru',
       'i-bank ┴шыaщэ (aтєoяыa>moscow         ru',
       'i-bank aшcє           >moscow         ru',
       'i-bank дom.ru         >moscow         ru',
       'i-bank mtc фикcиpoвa

Будем искать транзакции где клиенты покупали кофе/посещали кофейни.\
Так как мы не знаем, что конкретно покупал клиент в точке будет подбирать те точки, где больше половины ассортимента это кофе.\ 
При этом будет понимать, что в нашу выборку могут попасть и те клиенты, кто "зашел в Старбакс и купил бутылку воды".\
Будем отбирать в три стадии:\
1. Транзакции где явно есть отдельностоящие слова типа кофе, кофейня или название популярного места 
2. Транзакции где в символах встречается что-то похожее на "коф" + ручной отбор таких транзакций 
3. Трандакции где в символах встречается название популярного места где продают кофе, но в названии самого места нет кофе. Например: Starbucks 




1. Транзакции где явно есть отдельностоящие слова типа кофе, кофейня или название популярного места 

In [159]:
list_exact_coffee = df_transactions[
    (df_transactions['merchant_name'].str.contains('coffee', case=False,regex=False)) | 
    (df_transactions['merchant_name'].str.contains(r'\b(?:coffe|cofe|kofe|koffe|koffee|coffe|cofee|kofejnya|kofeiynya|kofeynya|kofeinya|kofeinia|kofeshop|kofeinaya|kofejnaya|cofix|kofein|onepricecoffe|koffein|kofeynyy)\b', case=False,regex=True)) 
]['merchant_name'].unique().tolist()

In [160]:
len(list_exact_coffee)

961

2. Транзакции где в символах встречается что-то похожее на "коф" + ручной отбор таких транзакций 

In [161]:
df_transactions_prob_coffee = df_transactions[
    (df_transactions['merchant_name'].str.contains('cof|kof', case=False,regex=True)) &
    (~df_transactions['merchant_name'].isin(list_exact_coffee)) 
    ]['merchant_name'].unique()

Выгрузим файл мерчанты которые похожи на кофе, чтобы разметить их вручную. Их чуть больше 200, что не так много. 

In [162]:
pd.DataFrame(df_transactions_prob_coffee).to_excel('df_transactions_prob_coffee.xlsx')

In [ ]:
# тут я поработал с Excel файлом
# руками разметил мерченты которые явно относятся к кофе 

In [165]:
df_transactions_prob_coffee = pd.read_excel('df_transactions_prob_coffee_fixed.xlsx',index_col = 0)

In [169]:
df_transactions_prob_coffee.columns = ['merchant_name', 'coffee']


In [175]:
#добавляем к списку наших кофейных мерчантов тех, которых мы отобрали вручную 

list_exact_coffee = list_exact_coffee + df_transactions_prob_coffee[df_transactions_prob_coffee['coffee'] == 1]['merchant_name'].to_list()

In [176]:
len(list_exact_coffee)

1113

3. Трандакции где в символах встречается название популярного места где продают кофе, но в названии самого места нет кофе. \
Например: Starbucks 
Погуглил в интернете кофейни и магазины кофе и искал то что без "Коф" в названии
Также пользовался вот этим списком

In [222]:
list_exact_coffee = list_exact_coffee + df_transactions[df_transactions['merchant_name'].str.contains('starbucks|bodryj den|shokoladnitsa|doubleb|bloom n brew|cooperative chernyi', case=False,regex=True)]['merchant_name'].to_list()

4. Транзакции в которых встречаютстя кофейны слова типа latte, cappuccino 

In [225]:
list_exact_coffee = list_exact_coffee + df_transactions[df_transactions['merchant_name'].str.contains('capuchino|cappuccino|latte|robusta|espresso', case=False,regex=True)]['merchant_name'].to_list()

In [228]:
len(list_exact_coffee)

1314

In [227]:
#удаляем дубликаты из листа мерчантов
list_exact_coffee = list(dict.fromkeys(list_exact_coffee))

Размечаем транзакции в которых есть покупки кофе 

In [252]:
df_transactions['coffee_transaction'] = np.where(df_transactions['merchant_name'].isin(list_exact_coffee), True, False)

Делаем компактный Датафрейм с кофейными транзакциями размеченный по месяцам

In [320]:
date = pd.to_datetime(df_transactions['tr_datetime'])
df_transactions['month'] = date.dt.month

In [321]:
df_transactions_short = df_transactions[df_transactions['coffee_transaction'] == True][['customer_id','month','card_id']] 

In [356]:

df_transactions['customer_id'].unique().shape[0]

5370

Рисуем пивот таблицу где напротив каждого месяца будет количество транзакций 

In [339]:
df_coffee_pivot =  df_transactions_short.pivot_table(columns=['month'],index=['customer_id'],aggfunc= 'count',fill_value=0).reset_index()

In [350]:
df_coffee_pivot.shape[0]

1151

In [340]:

#Фиксим колонки в табличке 
#Исправляем табличку 
df_coffee_pivot.columns = [' '.join(map(str, col)).strip() for col in df_coffee_pivot.columns.values]

In [344]:
df_coffee_pivot.columns = ['customer_id', '8', '9', '10']

In [349]:
df_coffee_pivot[
    ((df_coffee_pivot['8']>1) & (df_coffee_pivot['9']>1)) |
    ((df_coffee_pivot['8']>1) & (df_coffee_pivot['10']>1)) |
    ((df_coffee_pivot['9']>1) & (df_coffee_pivot['10']>1))]

,customer_id,8,9,10
5,01F4759D39152AD450EB146328553FF4,4,2,0
9,02742357300A71889E018AD35941DA09,2,2,0
12,03FF8D5865069ED9A1810F0464AD0FBA,11,5,5
13,0405815580CA4D1A190D4E04DA422485,10,3,7
14,040CA8756FC68D5D173495B66F8B0C28,3,3,2
...,...,...,...,...
1137,FDF18BD8668193BB7EA965A1353E1FD0,1,3,2
1138,FE257FE01D64EDA62D336934C01756B7,6,0,2
1142,FE9698DE39B77B5DB9EAFE1D9B13CE67,2,4,0
1143,FEB266B829BAB63CA5CD60CB73B74877,5,3,3
